In [1]:
import os
import handlePhs as hp
import numpy as np
import plotSig as ps
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from skimage.feature import graycomatrix, graycoprops
from rich.progress import track

In [2]:
def to255(array,length,width):

    ymax = 255
    ymin = 0
    xmax = max(map(max,array))
    xmin = min(map(min,array))
    
    for i in range(length):
        for j in range(width):
            array[i][j] = round(((ymax-ymin)*(array[i][j]-xmin)/(xmax-xmin))+ymin)
    
    return array

In [3]:
fileList='./filelist'
# headerFile='./data1/20180102-20180108.phs.ovr.rsc'
headerFile='./header'
result='./result_09'
if not os.path.exists(result):
    os.makedirs(result)
sumOfRatio=0.9

In [4]:
header, phsStack, fileNum=hp.readFile(fileList,headerFile)

read file list: ./filelist
read header file: ./header
with: 4865  length: 3241


Output()

In [5]:
pca = PCA()
pca.fit(phsStack)
ratio=pca.explained_variance_ratio_

In [6]:
print(ratio)

[5.31887107e-01 1.25983583e-01 5.65184616e-02 4.33150790e-02
 3.55924144e-02 2.71235114e-02 2.05864058e-02 1.69017984e-02
 1.48407447e-02 1.26312448e-02 1.01180973e-02 9.30762400e-03
 8.02959445e-03 7.70176450e-03 6.79019220e-03 5.89675305e-03
 4.84367170e-03 4.75002395e-03 4.23426039e-03 4.01267657e-03
 3.80456442e-03 3.60348708e-03 3.33161525e-03 3.18674784e-03
 2.99012673e-03 2.69146668e-03 2.59268810e-03 2.48618782e-03
 2.33499661e-03 2.20493521e-03 2.04253604e-03 1.75983423e-03
 1.67219332e-03 1.55163211e-03 1.46994339e-03 1.36057845e-03
 1.27781399e-03 1.25327609e-03 1.13704937e-03 1.07446163e-03
 9.86621335e-04 8.76995917e-04 8.39475967e-04 7.54407794e-04
 6.53128146e-04 5.77371716e-04 4.20856486e-04]


In [7]:
sum=0
for i in range(len(ratio)):
    sum+=ratio[i]
    if(sum>sumOfRatio):
        print(i)
        print(sum)
        break


11
0.9048060714849041


In [8]:
numOfPCA=i

In [9]:
phsMean=phsStack.mean(axis=0)
phsSubMeanStack=phsStack-phsMean

In [10]:
ica = FastICA(whiten='unit-variance')
S_ = ica.fit_transform(phsSubMeanStack)  # Reconstruct signals
A_ = ica.mixing_  # Get estimated mixing matrix
S_ = S_.astype('float32')
A_ = A_.astype('float32')

In [11]:
print(S_.shape)
print(A_.shape)

(15767465, 47)
(47, 47)


In [12]:
numOfS=S_.shape[1]

In [13]:
contrast=np.empty(numOfS)
dissimilarity=np.empty(numOfS)
energy=np.empty(numOfS)
correlation=np.empty(numOfS)
homogeneity=np.empty(numOfS)

In [14]:
for i in range(S_.shape[1]):
    SImageLine=S_[:,i]
    SImage=np.round(SImageLine.reshape([header.length,header.width])).astype('int64')
    gray_image=to255(SImage,header.length,header.width)

    d = [5]  # 距离
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # 角度

    # 计算GLCM
    glcm = graycomatrix(gray_image, distances=d, angles=angles, levels=256, symmetric=True, normed=True)

    # 计算GLCM特征
    contrast[i] = np.mean(graycoprops(glcm, 'contrast'))
    dissimilarity[i] = np.mean(graycoprops(glcm, 'dissimilarity'))
    energy[i] = np.mean(graycoprops(glcm, 'energy'))
    correlation[i] = np.mean(graycoprops(glcm, 'correlation'))
    homogeneity[i] = np.mean(graycoprops(glcm, 'homogeneity'))

    print(i)
    # 输出特征值
    print("Contrast:", contrast[i])
    print("Dissimilarity:", dissimilarity[i])
    print("Energy:", energy[i])
    print("Correlation:", correlation[i])
    print("Homogeneity:", homogeneity[i])

0
Contrast: 42.92885098400763
Dissimilarity: 2.0658394861548084
Energy: 0.7431566375162898
Correlation: 0.852780483244765
Homogeneity: 0.8657314225928442
1
Contrast: 53.2760050926594
Dissimilarity: 2.521628738043738
Energy: 0.7312924682391995
Correlation: 0.8827723111918212
Homogeneity: 0.8606780908787259
2
Contrast: 55.940347705524886
Dissimilarity: 2.748333385504882
Energy: 0.6959725650287543
Correlation: 0.7161541697964714
Homogeneity: 0.8194476576765075
3
Contrast: 58.17494341893957
Dissimilarity: 2.755184437500626
Energy: 0.7022352102341544
Correlation: 0.7269383545824409
Homogeneity: 0.8235969535932186
4
Contrast: 62.719098163236865
Dissimilarity: 2.9106510246267554
Energy: 0.7016876060004094
Correlation: 0.6616467805789773
Homogeneity: 0.8149799017923423
5
Contrast: 54.27569230884516
Dissimilarity: 2.5931764717093437
Energy: 0.7360900712863343
Correlation: 0.8350955246775171
Homogeneity: 0.846392467680634
6
Contrast: 39.602025823864295
Dissimilarity: 2.205097090078152
Energy: 0.

In [15]:
def normalize_array(arr, epsilon=1e-10):
    """
    Normalize the input array to the range [0, 1].
    
    Parameters:
        arr (array-like): Input array.
    
    Returns:
        array: Normalized array.
    """
    arr = np.array(arr)
    min_val = np.min(arr)
    max_val = np.max(arr)
    if min_val == max_val:
        return np.full_like(arr, epsilon)
    else:
        return (arr - min_val) / (max_val - min_val) * (1 - 2 * epsilon) + epsilon

In [16]:
contrastNor=normalize_array(contrast)
dissimilarityNor=normalize_array(dissimilarity)
# energyNor=normalize_array(energy)
# correlationNor=normalize_array(correlation)
one_contrastNor=1-contrastNor
one_dissimilarityNor=1-dissimilarityNor

In [17]:
print(one_contrastNor)
print(one_dissimilarityNor)

[6.17876527e-01 5.03780249e-01 4.74401005e-01 4.49760504e-01
 3.99652897e-01 4.92756872e-01 6.54560852e-01 1.00000008e-10
 6.51096143e-01 5.52911165e-01 4.41641708e-01 4.77968954e-01
 5.35010712e-01 3.66953427e-01 4.90892511e-01 6.09237013e-01
 3.97689339e-01 4.67304492e-01 3.51917245e-01 1.00000000e+00
 1.47336511e-01 4.55862843e-01 4.46774369e-01 6.13079701e-01
 2.80347229e-01 7.38908311e-01 8.92796091e-01 8.23236467e-01
 4.25117207e-01 6.94496211e-01 6.30959705e-01 3.21216577e-01
 6.52605625e-01 5.88768302e-01 4.03696202e-01 5.59227451e-01
 6.11433679e-01 5.13522906e-01 6.41149108e-01 3.54104937e-01
 4.24179196e-01 4.16624860e-01 9.28520318e-01 2.70130165e-01
 4.42386833e-01 4.41230232e-01 3.67827839e-01]
[5.66959908e-01 4.10876535e-01 3.33242352e-01 3.30896234e-01
 2.77657267e-01 3.86375273e-01 5.19271647e-01 1.00000008e-10
 5.79887133e-01 4.59656923e-01 3.39208354e-01 3.35399222e-01
 4.17887586e-01 2.69383760e-01 3.52900135e-01 4.48690913e-01
 2.77231556e-01 3.54934695e-01 2.51531

In [18]:
GLCM=one_contrastNor*one_dissimilarityNor*energy*correlation

In [19]:
print(np.size(GLCM))

47


In [20]:
def top_n_indices(arr, n):
    """
    Find the indices of the top n elements in the array (sorted in descending order).
    
    Parameters:
        arr (array-like): Input array.
        n (int): Number of top indices to return.
    
    Returns:
        array: Array containing the indices of the top n elements.
    """
    arr = np.array(arr)
    indices = np.argsort(arr)[::-1][:n]
    return indices

In [21]:
topGLCMIndex=top_n_indices(GLCM, numOfPCA)

In [22]:
print(topGLCMIndex)

[19 26 42 27  0 30  8  6 36 32 29]


In [23]:
# newS = np.delete(S_,obj=topGLCMIndex,axis=1)
# newA = np.delete(A_,obj=topGLCMIndex,axis=1)
newS = S_[:,topGLCMIndex]
newA = A_[:,topGLCMIndex]
print(newS.shape)
print(newA.shape)

(15767465, 11)
(47, 11)


In [24]:
newPhsStackMean = np.dot(newS, newA.T)
newPhsStack = newPhsStackMean + ica.mean_.astype('float32')

In [25]:
print(newPhsStack.shape)

(15767465, 47)


In [26]:
for i in track(range(S_.shape[1]), description="writing S files", total=S_.shape[1]):
    hp.writePhs(S_[:, i], result+"/S_"+str(i)+".sig")

Output()

In [27]:
for i in track(range(A_.shape[1]), description="writing A files", total=A_.shape[1]):
    hp.writePhs(A_[:, i], result+"/A_"+str(i)+".sig")

Output()

In [28]:
f = open(fileList, 'r')
i=0
for line in track(open(fileList), description="writing cor files", total=fileNum):
        line = f.readline()
        phs = hp.readPhs(line.strip())
        index = np.where(phs == 0)
        newPhsStack[index, i] = 0
        line = os.path.split(line)[1].strip()
        hp.writePhs(newPhsStack[:, i], result+"/"+line+".ICAcorr")
        i = i+1

Output()

In [29]:
ps.saveSubFig(newPhsStack, header, result+'/output.png')
ps.saveSubFig(S_, header, result+'/S_all.png')
ps.saveLineFig(A_,result+'/A_all.png')

In [30]:
ps.saveSubFig(newS, header, result+'/newS_all.png')